### BIG DATA ASSIGNMENT 4

### VIKRAM SARVAGYAM - 202318034

In [17]:
pip install pyspark

In [18]:
import numpy as np
from pyspark import SparkContext

In [19]:
sc_nums = SparkContext()

# Setting the seed for consistent results
np.random.seed(27)
# Generating random numbers
nums = np.random.randint(0, 11, 100)
print(nums)

# Parallelize into a spark RDD
nums_rdd = sc_nums.parallelize(nums)
print()
# Calculating the frequency of numbers using MapReduce
num_freq_pair = nums_rdd.map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b)
# Assigning to a list
frequencies = num_freq_pair.collect()
# Sorting the list in ascending order and printing the frequency of each number
sorted_list = sorted(frequencies, key=lambda x: x[0])
for i in sorted_list:
  print(i[0], 'occurs', i[1], 'times in the randomly generated list')
sc_nums.stop()


[ 3  8  8  8  0  5  8  9  1  2 10  1 10  1  4  6  3  3 10  0  5  3  3 10
  6  7  2  7  8  5  7  1  2  0  3  3  5 10  3  5  0  7  7  4  1  1  7  0
  7  8  4  9  0  4  7  0  9  2  2  3  0  1  5  9  5  1  3 10  9  6  5  7
  3  4 10  4  4  2  0  2  4  9  8  0  7  5  6  2  2  0 10  0  3 10 10  5
  6  3  7  6]

0 occurs 12 times in the randomly generated list
1 occurs 8 times in the randomly generated list
2 occurs 9 times in the randomly generated list
3 occurs 13 times in the randomly generated list
4 occurs 8 times in the randomly generated list
5 occurs 10 times in the randomly generated list
6 occurs 6 times in the randomly generated list
7 occurs 11 times in the randomly generated list
8 occurs 7 times in the randomly generated list
9 occurs 6 times in the randomly generated list
10 occurs 10 times in the randomly generated list


In [20]:
sc_word = SparkContext()

# Load text8 file into an RDD
text8_rdd = sc_word.textFile('/content/text8')

# Make a flattened RDD by flattening each line
words_rdd = text8_rdd.flatMap(lambda line: line.split())

total_words = words_rdd.collect()

# Using MapReduce to get the frequency of all words, regardless of whether they contain a or not
word_frequencies_rdd = words_rdd.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)

# Filter the RDD to get frequencies of words containing the letter 'a'
words_with_a_rdd = word_frequencies_rdd.filter(lambda pair: 'a' in pair[0])
word_frequencies_with_a = words_with_a_rdd.collect()

# Sort the words by frequency in ascending order
sorted_frequencies = sorted(word_frequencies_with_a, key=lambda x: x[1])

# Print the frequencies of words containing the letter 'a'
counter = 0
for word, frequency in sorted_frequencies:
    counter += 1
    print(f"Word {counter}: \"{word}\" occurs {frequency} times")

# Calculate the percentage of words that contained the letter 'a'
print('\n\n', round(len(sorted_frequencies) / len(total_words) * 100, 3), '% of words contain the letter \'a\'')

# Stop the SparkContext
sc_word.stop()

Streaming output truncated to the last 5000 lines.
Word 72241: "medications" occurs 42 times
Word 72242: "taboo" occurs 42 times
Word 72243: "ammon" occurs 42 times
Word 72244: "marc" occurs 42 times
Word 72245: "packet" occurs 42 times
Word 72246: "operas" occurs 42 times
Word 72247: "dangers" occurs 42 times
Word 72248: "rapper" occurs 42 times
Word 72249: "accordion" occurs 42 times
Word 72250: "cha" occurs 42 times
Word 72251: "constantius" occurs 42 times
Word 72252: "alamanni" occurs 42 times
Word 72253: "minneapolis" occurs 42 times
Word 72254: "craps" occurs 42 times
Word 72255: "emphasizes" occurs 42 times
Word 72256: "facial" occurs 42 times
Word 72257: "fluctuations" occurs 42 times
Word 72258: "emirates" occurs 42 times
Word 72259: "heracles" occurs 42 times
Word 72260: "appealed" occurs 42 times
Word 72261: "circulated" occurs 42 times
Word 72262: "aquinas" occurs 42 times
Word 72263: "vita" occurs 42 times
Word 72264: "repeating" occurs 42 times
Word 72265: "abandon" occu

In [21]:
from pyspark.sql import SparkSession

In [22]:
# Create a SparkSession
spark = SparkSession.builder.appName("IrisCorrelation").getOrCreate()

# Reading the file and printing the top 20 rows and schema
df_iris = spark.read.json('/content/iris.json', multiLine=True)
df_iris.show()
df_iris.printSchema()

+-----------+----------+-----------+----------+-------+
|petalLength|petalWidth|sepalLength|sepalWidth|species|
+-----------+----------+-----------+----------+-------+
|        1.4|       0.2|        5.1|       3.5| setosa|
|        1.4|       0.2|        4.9|       3.0| setosa|
|        1.3|       0.2|        4.7|       3.2| setosa|
|        1.5|       0.2|        4.6|       3.1| setosa|
|        1.4|       0.2|        5.0|       3.6| setosa|
|        1.7|       0.4|        5.4|       3.9| setosa|
|        1.4|       0.3|        4.6|       3.4| setosa|
|        1.5|       0.2|        5.0|       3.4| setosa|
|        1.4|       0.2|        4.4|       2.9| setosa|
|        1.5|       0.1|        4.9|       3.1| setosa|
|        1.5|       0.2|        5.4|       3.7| setosa|
|        1.6|       0.2|        4.8|       3.4| setosa|
|        1.4|       0.1|        4.8|       3.0| setosa|
|        1.1|       0.1|        4.3|       3.0| setosa|
|        1.2|       0.2|        5.8|       4.0| 

In [23]:
# Calculating correlation coefficient
correlation = df_iris.stat.corr("petalLength", "petalWidth")
print('Pearson correlation between Petal Length and Petal Width:', correlation)

Pearson correlation between Petal Length and Petal Width: 0.962865431402796


In [24]:
# Creating view to perform SQL operations
df_iris.createTempView("iris")

In [25]:
# Show the sepal length, sepal width and species of the flowers which have petal length greater than 1.4
spark.sql("SELECT sepalLength, sepalWidth, species FROM iris WHERE petalLength >= 1.4").show()

+-----------+----------+-------+
|sepalLength|sepalWidth|species|
+-----------+----------+-------+
|        5.1|       3.5| setosa|
|        4.9|       3.0| setosa|
|        4.6|       3.1| setosa|
|        5.0|       3.6| setosa|
|        5.4|       3.9| setosa|
|        4.6|       3.4| setosa|
|        5.0|       3.4| setosa|
|        4.4|       2.9| setosa|
|        4.9|       3.1| setosa|
|        5.4|       3.7| setosa|
|        4.8|       3.4| setosa|
|        4.8|       3.0| setosa|
|        5.7|       4.4| setosa|
|        5.1|       3.5| setosa|
|        5.7|       3.8| setosa|
|        5.1|       3.8| setosa|
|        5.4|       3.4| setosa|
|        5.1|       3.7| setosa|
|        5.1|       3.3| setosa|
|        4.8|       3.4| setosa|
+-----------+----------+-------+
only showing top 20 rows



In [26]:
spark.stop()